In [14]:
import sys
sys.path.append("../")

In [ ]:
#!pip install selenium

In [15]:
%load_ext autoreload
%autoreload 2

In [16]:
import requests
import json
import time

from pprint import pprint as print_
from collections import OrderedDict
import numpy as np
from tqdm import tqdm_notebook as tqdm

from isanlp_srl_framebank.pipeline_default import PipelineDefault

In [ ]:
from time import sleep

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

In [64]:
!curl -d "{\"yandexPassportOauthToken\":\"y0_AgAAAAAiUyyhAATuwQAAAADS3TX_f0yBTjPTRMCBygtxvEC5suQjajg\"}" "https://iam.api.cloud.yandex.net/iam/v1/tokens" -o iamToken.txt


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   341  100   252  100    89   1296    457 --:--:-- --:--:-- --:--:--  1766


In [65]:
import json
with open('iamToken.txt') as f:
    d = json.loads(f.read())
    IAM_TOKEN = d['iamToken']

In [66]:
# IAM_TOKEN = 't1.9euelZqaz5DGjI6Py5bLnoqTlo_Oju3rnpWaj46Nkc6ejZaLyZvOmZqWj53l8_dIF2Zk-e9eDic9_t3z9whGY2T5714OJz3-.1UJE5innTD10cDR2-7YbCzvwem5VKOvoIQCksQwLbJAA7oAdm4OpX9lgQyC6cosbkLvAMqxKaFT1SfQlDXTlBg'
folder_id = 'b1gs2kplab3ve997i80t'
IAM_TOKEN

't1.9euelZqZyZGXxpWSkJudjYqWzJSKnO3rnpWaj46Nkc6ejZaLyZvOmZqWj53l9Pc6I1Rj-e9RRU383fT3elFRY_nvUUVN_A.WPo9dx1BCSQpy4_p66KPc0ZZiU-i4jHc0pruaZhoRb1CUbvs56qg5TvmTOP2wsZzBV8ZPddjWRHnV1Q6keLDBA'

In [14]:
# docker run --rm -p 3333:3333 inemo/isanlp
# docker run --rm --shm-size=1024m -ti -p 3334:9999 inemo/syntaxnet_rus server 0.0.0.0 9999
# docker run --rm -p 3335:3333 inemo/isanlp_srl_framebank

In [20]:
from isanlp_srl_framebank.pipeline_default import PipelineDefault  
#
ppl = PipelineDefault(address_morph=('localhost', 3333),
                      address_syntax=('localhost', 3334),
                      address_srl=('localhost', 3335))
res = ppl('Мы поехали на дачу.')

In [146]:
%%time

language = languages[0]
url = 'https://beta.apertium.org/index.eng.html#analysis?aLang=' + language + '&aQ='    
browser = webdriver.Chrome()
browser.get(url)


CPU times: total: 31.2 ms
Wall time: 4.61 s


In [159]:
class Model_srl1:
    def __init__(self, RUS_SENTENCE=None):
        self.RUS_SENTENCE = RUS_SENTENCE
        self.ppl = PipelineDefault(address_morph=('localhost', 3333),
                      address_syntax=('localhost', 3334),
                      address_srl=('localhost', 3335))
        self.language_code_dict =  {
        'татарский': 'tt',
        'казахский' : 'kk',
        'башкирский':'ba',
        'киргизский' : 'ky'
        }

    def translation(self, texts, source_language='tt', target_language='ru'):
        #print('texts', texts)
        body = {
            "targetLanguageCode": target_language,
            "texts": texts,
            "folderId": folder_id,
            "sourceLanguageCode":  source_language
        }

        headers = {
            "Content-Type": "application/json",
            "Authorization": "Bearer {0}".format(IAM_TOKEN)
        }
        url = 'https://translate.api.cloud.yandex.net/translate/v2/translate'
        response = requests.post(url,
            json = body,
            headers = headers
        )
        if response.status_code != 200:
            print('Ожидаю 0.5 секунды...')  
            time.sleep(0.5)
            response = requests.post(url,
            json = body,
            headers = headers
            )
        d = json.loads(response.text)
        #print(texts, d)
        translations = d['translations']
        return [t['text'] for t in translations]
    
    def drop_punkt(self, word):
        punc = '''!()-[]{};:"\,<>./?@#$%^&*_~'''
        for p in punc:
            word = word.replace(p,'')
        return word
    
    def tokenize(self, text):
        words = text.split()
        return [self.drop_punkt(word) for word in words]
    
    def get_morph(self, text, language):
        lang_field = browser.find_element(By.XPATH, '/html/body/div/div[1]/div[1]/form/div[1]/div/select')
        lang_field.send_keys(language)

        text_field = browser.find_element(By.XPATH, '/html/body/div/div[1]/div[1]/form/div[2]/div/textarea').clear()
        text_field = browser.find_element(By.XPATH, '/html/body/div/div[1]/div[1]/form/div[2]/div/textarea')

        text = drop_punc(text)
        text_field.send_keys(text)

        button = browser.find_element(By.XPATH, '/html/body/div/div[1]/div[1]/form/div[3]/div/button')
        button.click()
        sleep(1)

        html = browser.page_source
        soup = BeautifulSoup(html, 'lxml')
        translation = soup.find_all('td',class_="text-left")
        words = []
        for word in translation:
            word = word.text.split()
            for i, w in enumerate(word):
                if w == '↤':
                    break
            word = word[:i]
            for w in word:
                words.append(w)
        return words

    
    # def get_morph_list(self, words):
    #     result = []
    #     for i, word in enumerate(words):
    #         result.append(get_morph(word))
    #     return result
    
    def print_roles(self, lemma, role_annot, targetLanguage=None):
        word_role = {}
        for sent_num, ann_sent in enumerate(role_annot):
            for event in ann_sent:
                lemma_pred = lemma[sent_num][event.pred[0]]
                if targetLanguage is not None:
                    trg_lang = self.language_code_dict[targetLanguage]
                    src_lang = 'ru'
                    #print('=====Pred: {}'.format(self.translation(lemma_pred, src_lang, trg_lang)[0]))
                    word_role[self.translation(lemma_pred, src_lang, trg_lang)[0]] = 'Pred'
                else:
                    print('=====Pred: {}'.format(lemma_pred))
                for arg in event.args:
                    lemma_arg  = lemma[sent_num][arg.begin]
                    if targetLanguage is not None:
                        #print('Arg({}): {}'.format(arg.tag, self.translation(lemma_arg, src_lang, trg_lang)[0]))
                        word_role[self.translation(lemma_arg, src_lang, trg_lang)[0]] = 'Arg({})'.format(arg.tag)
                    else:
                        print('Arg({}): {}'.format(arg.tag, lemma_arg))
        return word_role
    
    def get_srl_turkic(self, rus_text, selected_language):
        # texts = ["Без дачага киттек.", "Балалар өйдән кача һәм урманда югалып кала.","Әни рамны юды. Әти машинаны йөртте."]
        # словарь - лемма:роль
        # src_code = self.language_code_dict[selected_language]
        # translation = translate(text, source_language=src_code )
        # # print(translations)
        # print(f'исходный  текст на {selected_language[:-2]}ом', end='\n\n')
        # rus_text = translation
        # print('переведенный текст на русском', end='\n\n')
        # print(rus_text)
        #print('get_srl_turkic_rus_text', rus_text)
        res = ppl(rus_text)
        # print(f'разметка семантических ролей на русском')
        # print_roles(res['lemma'], res['srl'])
        # print()
        # print(f'разметка семантических ролей на {selected_language[:-2]}ом')
        return self.print_roles(res['lemma'], res['srl'], selected_language)
        
        
    def predict(self, input_sentences, language_ru_name):
        
        languages_rus__eng = {'татарский':'Tatar', 'казахский':'Kazakh', 'киргизский':'Kyrgyz', 'башкирский':'Bashqort'}
        for input_sentence in input_sentences:
            #перевод
            rus_sentence = self.translation(input_sentence, source_language=self.language_code_dict[language_ru_name])[0]
            # токенизация исходного предложения
            #input_words = tokenize(input_sentence)
            # стемминг слов исходного предложения
            stem_input_words = self.get_morph(input_sentence, languages_rus__eng[language_ru_name])
            #print('stems',stem_input_words)
            # получить список словарей лемма - роль
            word_roles = self.get_srl_turkic(rus_sentence, language_ru_name)
            #print('word roles',word_roles)
            # применить к леммам стемминг
            stem_role = {self.get_morph(k, languages_rus__eng[language_ru_name])[0]: v for k, v in word_roles.items()}
            #print('stem role', stem_role)
            # разметить всю последовательность по словарям полученным выше
            result = {}
            for stem_input_word in stem_input_words:
                if stem_input_word in stem_role:
                    result[stem_input_word] = stem_role[stem_input_word]
                else:
                    result[stem_input_word] = 'O'
            result_input = {}
            for k, v in zip(self.tokenize(input_sentence), result.values()):
                result_input[k] = v
            print(result_input)

In [160]:
srl = Model_srl1()

In [ ]:
%%time
with open('text_tt.txt', encoding='utf-8') as f:
    texts_tt = f.readlines()
selected_language = 'татарский'
result = srl.predict(texts_tt, selected_language)
result

{'Петр': 'O', 'дачага': 'Arg(конечная точка)', 'барыр': 'Pred', 'өчен': 'O', 'машина': 'Arg(пациенс)', 'сатып': 'O', 'алган': 'Pred'}
{'Петр': 'O', 'алманы': 'Arg(пациенс)', 'Иваннан': 'O', '5': 'O', 'сумга': 'O', 'сатып': 'O', 'алган': 'Pred'}
{'Без': 'Arg(субъект перемещения)', 'дачага': 'Arg(конечная точка)', 'киттек': 'O'}
{'Балалар': 'Arg(субъект перемещения)', 'өйдән': 'O', 'кача': 'Pred', 'һәм': 'O', 'урманда': 'Arg(место)', 'югалып': 'O', 'кала': 'O'}
